<a href="https://colab.research.google.com/github/bmanav26/Coronary-Artery-disease-prediction/blob/main/Optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install tensorflow

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121, ResNet50, InceptionV3, VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, AdamW, Adadelta, Adafactor, Nadam, SGD, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Flatten, Dense, Dropout

In [ ]:
dataset_path = "/content/drive/MyDrive/SPECT_MPI_Dataset"
batch_size = 32
epochs=50

Defining decaying learning rate function

In [ ]:
def lr_schedule(epoch):
    # Initial learning rate
    initial_lr = 0.0001

    # Learning rate decay factor
    decay_factor = 0.9

    # Learning rate decay every 'n' epochs
    decay_freq = 5

    # Calculate the new learning rate
    lr = initial_lr * (decay_factor ** (epoch // decay_freq))

    return lr

**Data Augmentation Function**

In [ ]:
data_generator = ImageDataGenerator(
    rescale=1.0 / 255.0, # this performs min-max normalization
    shear_range=0.20,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2
)

**Applying min-max normalization and data augmentation on given image dataset**

In [ ]:
train_generator = data_generator.flow_from_directory(
    os.path.join(dataset_path, 'TrainSet'),
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'  # Adjust as needed
)

Found 128 images belonging to 2 classes.


In [ ]:
test_generator = ImageDataGenerator(rescale=1.0 / 255.0).flow_from_directory(
    os.path.join(dataset_path, 'TestSet'),
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'  # Adjust as needed
)

Found 32 images belonging to 2 classes.


In [ ]:

validate_generator = data_generator.flow_from_directory(
    os.path.join(dataset_path, 'ValidationSet'),
    target_size=(256, 256),
    batch_size=batch_size,
    class_mode='binary'  # Adjust as needed
)

Found 32 images belonging to 2 classes.


**Defining pre-trained CNN models on imagenet**

In [ ]:
base_model_densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model_inception = InceptionV3(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
lr_scheduler=LearningRateScheduler(lr_schedule)

**Function to train a model with a particular optimizer and return test results**

In [ ]:
def model_function(base_model, opt):
  model = Sequential([
      base_model,
      Flatten(),
      Dense(256, activation='relu'),
      Dropout(0.3),
      Dense(128, activation='relu'),
      Dropout(0.3),
      Dense(64, activation='relu'),
      Dropout(0.3),
      Dense(1, activation='sigmoid')
  ])
  model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
  model.fit(train_generator, validation_data=validate_generator, epochs=epochs, callbacks=[lr_scheduler])
  test_loss, test_accuracy = model.evaluate(test_generator)
  return test_loss, test_accuracy

**Defining optimizers' names**

In [ ]:
optimizer_names=["RMSprop", "Adadelta", "Nadam", "SGD", "AdamW", "Adam"]

**Storing Models in a List**

In [ ]:
models=[base_model_densenet, base_model_resnet, base_model_inception, base_model_vgg]
model_names=["DenseNet121", "ResNet50", "InceptionV3", "VGG16"]

In [ ]:
test_results=[]

In [ ]:
print(model_names)
print(optimizer_names)

['DenseNet121', 'ResNet50', 'InceptionV3', 'VGG16']
['RMSprop', 'Adadelta', 'Nadam', 'SGD', 'AdamW', 'Adam']


**Training all models for RMSprop**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], RMSprop(learning_rate=0.0001))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Training all models for Adadelta**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], Adadelta(learning_rate=0.0001))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Training all models for Nadam**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], Nadam(learning_rate=0.0001))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Training all models for SGD with Momentum**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], SGD(learning_rate=0.0001, momentum = 0.9))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Training all models for AdamW**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], AdamW(learning_rate=0.0001))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Training all models for Adam**

In [ ]:
for i in range(len(models)):
  print(model_names[i])
  test_loss, test_accuracy=model_function(models[i], Adam(learning_rate=0.0001))
  test_results.extend([model_names[i], test_loss, test_accuracy])

**Printing Test Results**

In [ ]:
for i, optimizer in enumerate(optimizer_names):
  print(optimizer)
  for j in range(i*12, i*12+12):
    if j%3==0:
      print()
      print(test_results[j])
    elif j%3==1:
      print("Test Loss: ",test_results[j])
    else:
      print("Test Accuracy: ", test_results[j])
  print("----------------------------------------------------------------------------")